In [1]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


### Load document into backend

In [2]:
DOCUMENT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
)

result: Result[str, str] = backend.load_project(DOCUMENT_PATH)
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

LoadProject: path = /data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz


2024-09-04 11:19:10.349 | INFO     | __main__:<module>:8 - loaded document /data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz successfully


loaded project in 39.4339 sec


### Define helper functionality to visualize forward and inverse image mapping

In [3]:
import os

from mynd.api.identifier import Identifier
from mynd.api.camera import StereoCollection

from mynd.backends import metashape as backend

from mynd.geometry import load_hitnet
from mynd.runtime import Environment, load_environment

from mynd.tasks.export_stereo_geometry import (
    ExportStereoTask,
    invoke_stereo_export_task,
)


def prepare_task_config(
    identifier: Identifier, collection: StereoCollection
) -> ExportStereoTask.Config:
    """Prepares a stereo export task config by creating setting up paths, creating directories,
    and loading a disparity model."""

    environment: Environment = load_environment().unwrap()

    model: HitnetConfig = load_hitnet(
        environment.resource_directory
        / Path("hitnet_models/hitnet_eth3d_720x1280.onnx")
    ).unwrap()

    # Root directories
    RANGE_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_ranges")
    NORMAL_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_normals")

    config = ExportStereoTask.Config(
        range_directory=RANGE_DIRECTORY / Path(f"{identifier.label}_ranges"),
        normal_directory=NORMAL_DIRECTORY / Path(f"{identifier.label}_normals"),
        model=model,
        calibrations=collection.calibrations,
        image_loaders=collection.image_loaders,
    )

    # Create export directories if they do not exist
    if not config.range_directory.exists():
        os.mkdir(config.range_directory)
    if not config.normal_directory.exists():
        os.mkdir(config.normal_directory)

    return config


def main():
    """Main function."""
    result: Result = backend.camera_services.request_stereo_data()

    if result.is_err():
        logger.error(result.err())

    response_data: dict[Identifier, list[StereoCollection]] = result.ok()

    for identifier, collections in response_data.items():

        for collection in collections:

            config: ExportStereoTask.Config = prepare_task_config(
                identifier, collection
            )
            result: Result[ExportStereoTask.Result, str] = invoke_stereo_export_task(
                config
            )

            if result.is_err():
                logger.error(result.err())


# ---------- Invoke main function ----------
main()

2024-09-04 11:19:10.728089696 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 20 Memcpy nodes are added to the graph tf2onnx for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
Loading images...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2728/2728 [01:17<00:00, 35.02it/s]
2024-09-04 11:20:29.085337931 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 20 Memcpy nodes are added to the graph tf2onnx for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
Loading images...:  82%|███████████████████████████████████████████████████████████████

KeyboardInterrupt: 